In [203]:
import pandas as pd
import numpy
import yfinance as yf
import matplotlib.pyplot as plt
from matplotlib.backends.backend_tkagg import FigureCanvasTkAgg
from matplotlib.figure import Figure
import numpy as np
import tkinter as tk
from tkinter import ttk

In [251]:
class bclass:
    
      
    def __init__(self, window):
        self.window = window
        
        
        #labels section -
        self.stock_input_label = Label(window, text='Enter stock symbol:')
        self.stock_input_label.place(x=20,y=40)

        self.example_label = Label(window, text = '(E.g. Netflix = NFLX, Microsoft = MFST)')
        self.example_label.place(x=270, y=40)

        self.short_input_label = Label(window, text='Enter Short-term Moving Average period:')
        self.short_input_label.place(x=20,y=90)

        self.long_input_label = Label(window, text='Enter Long-term Moving Average period:')
        self.long_input_label.place(x=20,y=140)


        #textfields section -
        self.stock_text = Entry(window)
        self.stock_text.place(x=135, y=40)

        self.short_text = Entry(window)
        self.short_text.place(x=250, y=90)

        self.long_text = Entry(window)
        self.long_text.place(x=250, y=140)
        
        self.button = Button(window, text = 'Enter', command = self.plot)
        self.button.place(x=200, y=200)

        
    def plot(self):
        
        stock_name = self.stock_text.get()
        short = int(self.short_text.get())
        long = int(self.long_text.get())
        
        
        stock_data = yf.Ticker(stock_name)
        df = stock_data.history(period='max')
        
        
        short_window = short
        long_window = long
        signals = pd.DataFrame(index = df.index)
        signals['signal'] = 0.0
        signals['short_mavg'] = df['Close'].rolling(window = short_window, min_periods=1
                                                   , center = False).mean()
        signals['long_mavg'] = df['Close'].rolling(window = long_window, min_periods=1
                                                   , center = False).mean()
        signals['signal'][short_window:] = np.where(signals['short_mavg'][short_window:]
                                                > signals['long_mavg'][short_window:], 1.0, 0.0)
        signals['positions'] = signals['signal'].diff()

        fig = Figure(figsize = (20,15))
        ax1 = fig.add_subplot(111, ylabel = 'Price in $')
        signals[['short_mavg', 'long_mavg']].plot(ax=ax1, lw=2.)
        ax1.plot(signals.loc[signals.positions == 1.0].index, signals.short_mavg
                [signals.positions == 1.0], '^', markersize=10, color='g')
        ax1.plot(signals.loc[signals.positions == -1.0].index, signals.short_mavg
                [signals.positions == -1.0], 'v', markersize=10, color='r')

        

        canvas = FigureCanvasTkAgg(fig, master=self.window)
        canvas.get_tk_widget().pack()
        canvas.draw()
           
    
window = Tk()
window.title('Moving SMA')
window.geometry('900x900')

start = bclass(window)

window.mainloop()